In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
raw_graph = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/graph.tsv")

In [4]:
raw_graph.collect()

['A\tB', 'A\tC', 'B\tD', 'C\tA', 'C\tB', 'C\tD', 'D\tC']

In [5]:
import re
import sys
from operator import add

In [6]:
def computeContribs(urls, rank):
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [7]:
def parseNeighbors(urls):
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [8]:
links = raw_graph.map(parseNeighbors)\
                 .distinct()\
                 .groupByKey()

In [9]:
links.collect()

[('C', <pyspark.resultiterable.ResultIterable at 0x7f936c81c518>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x7f936c81c588>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x7f936c81c5c0>),
 ('D', <pyspark.resultiterable.ResultIterable at 0x7f936c81c630>)]

In [13]:
ranks = links.map(lambda url_neighbours: (url_neighbours[0], 1))

In [14]:
ranks.collect()

[('C', 1), ('A', 1), ('B', 1), ('D', 1)]

In [15]:
links.join(ranks).collect()

[('A', (<pyspark.resultiterable.ResultIterable at 0x7f936c82a2b0>, 1)),
 ('B', (<pyspark.resultiterable.ResultIterable at 0x7f938468f9e8>, 1)),
 ('C', (<pyspark.resultiterable.ResultIterable at 0x7f938468fef0>, 1)),
 ('D', (<pyspark.resultiterable.ResultIterable at 0x7f938468f898>, 1))]

In [16]:
contribs = links.join(ranks).flatMap(lambda u: computeContribs(u[1][0], u[1][1]))

In [17]:
contribs.collect()

[('B', 0.5),
 ('C', 0.5),
 ('D', 1.0),
 ('A', 0.3333333333333333),
 ('B', 0.3333333333333333),
 ('D', 0.3333333333333333),
 ('C', 1.0)]

In [19]:
from operator import add

In [20]:
add(1, 2)

3

In [21]:
contribs.reduceByKey(add).collect()

[('B', 0.8333333333333333),
 ('A', 0.3333333333333333),
 ('C', 1.5),
 ('D', 1.3333333333333333)]

In [22]:
iterations = 3
for iteration in range(iterations):
    contribs = links.join(ranks).flatMap(lambda u: computeContribs(u[1][0], u[1][1]))
    ranks = contribs.reduceByKey(add)
    print("Iteration #{}".format(iteration))
    print("Current pagerank is:")
    print(sorted(ranks.collect()))

Iteration #0
Current pagerank is:
[('A', 0.3333333333333333), ('B', 0.8333333333333333), ('C', 1.5), ('D', 1.3333333333333333)]
Iteration #1
Current pagerank is:
[('A', 0.5), ('B', 0.6666666666666666), ('C', 1.5), ('D', 1.3333333333333333)]
Iteration #2
Current pagerank is:
[('A', 0.5), ('B', 0.75), ('C', 1.5833333333333333), ('D', 1.1666666666666665)]


In [ ]:
sc.stop()